In [1]:
# ============================================================
# 1. 기본 import + 디바이스 설정
# ============================================================
import os
import sys
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", device)


[INFO] Using device: cuda


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# [1] --- DataHandler 클래스 정의 (V2: Scaler + zfill) ---
# (Phase 2-A와 2-C가 합쳐진 최종 버전)
import pandas as pd
import numpy as np
import os
import sys

# ============================================================
# 2. 경로 설정 (프로젝트 구조에 맞게 필요시 수정)
# ============================================================

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table_v2.csv")
MASTER_TABLE_PATH = os.path.join(DATA_DIR, "final_master_table_v2.csv")

GPT2_PATH       = os.path.join(PROJECT_ROOT, "pretrained_models", "gpt2")
TIME_LLM_ROOT   = os.path.join(PROJECT_ROOT, "external", "time-llm")

if TIME_LLM_ROOT not in sys.path:
    sys.path.append(TIME_LLM_ROOT)


class DataHandler:
    """
    [V2] 표준화(Standardization)와 zfill(6)이 적용된 DataHandler.
    """
    
    def __init__(self, file_path, train_end_date='2022-12-31'):
        self.file_path = file_path
        self.train_end_date = pd.to_datetime(train_end_date)
        self.data_by_ticker = {}   # 원본 데이터
        self.scalers_by_ticker = {} # Ticker별 Scaler
        self.tickers = []
        
        self._load_and_process_data()
        self._fit_scalers()
        
    def _load_and_process_data(self):
        try:
            # 1. dtype=str로 읽기
            df = pd.read_csv(
                self.file_path, 
                parse_dates=['date'],
                dtype={'ticker': str} 
            )
            # 2. zfill(6)로 '0' 채우기
            df['ticker'] = df['ticker'].str.zfill(6)
            df = df.set_index('date')
            
            self.tickers = df['ticker'].unique()
            
            for ticker in self.tickers:
                ticker_df = df[df['ticker'] == ticker].copy()
                channel_cols = [col for col in ticker_df.columns if col not in ['ticker']]
                self.data_by_ticker[ticker] = ticker_df[channel_cols]
            
            print(f"[DataHandler V2] Success: Loaded {len(self.tickers)} tickers.")
            print(f"[DataHandler V2] Available tickers: {self.tickers}")

        except Exception as e:
            print(f"[DataHandler V2] Error loading data: {e}")

    def _fit_scalers(self):
        """
        [Data Leakage 방지] 훈련 데이터로만 Scaler를 학습(fit)
        """
        print(f"[DataHandler V2] Fitting scalers using data up to {self.train_end_date.date()}...")
        for ticker in self.tickers:
            train_data = self.data_by_ticker[ticker].loc[:self.train_end_date]
            if train_data.empty:
                print(f"  > Warning: No training data for {ticker}.")
                continue
            
            scaler = StandardScaler()
            scaler.fit(train_data) # 'fit'은 훈련 데이터로만!
            self.scalers_by_ticker[ticker] = scaler
        print("[DataHandler V2] Scalers fitted.")

    def get_scaled_data_by_ticker(self, ticker):
        """
        'transform'은 전체 데이터에 적용하여 표준화된 DF 반환
        """
        if ticker not in self.scalers_by_ticker:
            print(f"[DataHandler V2] Error: No scaler for {ticker}")
            return None
        
        original_data = self.data_by_ticker[ticker]
        scaler = self.scalers_by_ticker[ticker]
        
        scaled_data_np = scaler.transform(original_data)
        
        scaled_df = pd.DataFrame(
            scaled_data_np, 
            index=original_data.index, 
            columns=original_data.columns
        )
        return scaled_df

    def get_all_tickers(self):
        return self.tickers

print("[INFO] PROJECT_ROOT:", PROJECT_ROOT)
print("[INFO] DATA_DIR    :", DATA_DIR)
print("[INFO] MASTER_TBL  :", MASTER_TABLE_PATH)
print("[INFO] GPT2_PATH   :", GPT2_PATH)


[INFO] PROJECT_ROOT: /workspace/ship-ai
[INFO] DATA_DIR    : /workspace/ship-ai/data/processed
[INFO] MASTER_TBL  : /workspace/ship-ai/data/processed/final_master_table_v2.csv
[INFO] GPT2_PATH   : /workspace/ship-ai/pretrained_models/gpt2


In [3]:
# ============================================================
# 4. TimeLLM 모델 import
# ============================================================
try:
    import importlib
    import models.TimeLLM
    importlib.reload(models.TimeLLM)
    from models.TimeLLM import Model as TimeLLM
    print("[INFO] TimeLLM 모델 임포트 성공")
except Exception as e:
    print("[ERROR] TimeLLM import 실패:", e)
    raise

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] TimeLLM 모델 임포트 성공


In [4]:

# ============================================================
# 5. 슬라이딩 윈도우 함수 + Dataset 정의
# ============================================================
def create_sliding_windows(data, input_seq_len, output_seq_len):
    """
    DataFrame(2D: [time, features]) -> (X, y) 3D numpy 배열로 변환
    X: (N, input_seq_len, C)
    y: (N, output_seq_len, C)
    """
    data_np = data.values
    n_samples = len(data_np)
    X, y = [], []

    total_len = input_seq_len + output_seq_len
    for i in range(n_samples - total_len + 1):
        x_win = data_np[i : i + input_seq_len]
        y_win = data_np[i + input_seq_len : i + total_len]
        X.append(x_win)
        y.append(y_win)

    return np.array(X), np.array(y)


class ShipDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.FloatTensor(Y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [5]:
class Configs:
    def __init__(self):
        # 기본 세팅
        self.task_name = 'long_term_forecast'
        self.is_training = 1
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'

        # 데이터 차원
        self.seq_len   = 120
        self.label_len = 60
        self.pred_len  = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12

        # LLM 설정 (학습 때와 동일하게!)
        self.llm_model      = 'GPT2'
        self.llm_model_path = GPT2_PATH
        self.llm_dim    = 768
        self.llm_layers = 8

        # Patch 설정
        self.patch_len = 8
        self.stride    = 4

        # 모델 차원
        self.d_model = 512
        self.d_ff    = 512
        self.n_heads = 12
        self.dropout = 0.00

        # Prompt
        self.prompt_domain = 1
        self.content = (
            "Task: Forecast daily closing prices for Korean shipbuilding companies. "
            "Input Data: 12 channels including OHLC prices, trading volume, "
            "and macro-indicators such as Brent oil price, USD/KRW exchange rate, "
            "interest rate, and BDI (Baltic Dry Index). "
            "Context: Shipbuilding stocks are sensitive to oil prices and BDI. "
            "Analyze the 120-day trend, focusing on volatility and correlations, "
            "and predict the next 10 days."
        )

        # 기타
        self.embed   = 'timeF'
        self.freq    = 'd'
        self.factor  = 1
        self.moving_avg = 25
        self.e_layers = 2
        self.d_layers = 1
        self.top_k    = 5


In [6]:
configs = Configs()
model = TimeLLM(configs).to(device).float()

SAVE_PATH = os.path.join(PROJECT_ROOT, "models", "ship_time_llm_tmp6_ft_h3_e5.pth")
state = torch.load(SAVE_PATH, map_location=device)
model.load_state_dict(state)
print("[LOAD] Loaded weights from", SAVE_PATH)

# 기존보다 살짝 낮은 lr로
LEARNING_RATE = 3e-5
ACCUM_STEPS = 8
EPOCHS = 8


optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

criterion = nn.MSELoss()

print("✅ Loaded model from:", SAVE_PATH)


[LOAD] Loaded weights from /workspace/ship-ai/models/ship_time_llm_tmp6_ft_h3_e5.pth
✅ Loaded model from: /workspace/ship-ai/models/ship_time_llm_tmp6_ft_h3_e5.pth


In [9]:
data_handler = DataHandler(MASTER_TABLE_PATH, train_end_date='2022-12-31')


[DataHandler V2] Success: Loaded 6 tickers.
[DataHandler V2] Available tickers: ['010140' '010620' '329180' '042660' '443060' '009540']
[DataHandler V2] Fitting scalers using data up to 2022-12-31...
  > Warning: No training data for 329180.
  > Warning: No training data for 443060.
[DataHandler V2] Scalers fitted.


In [10]:
import pandas as pd
import torch
import numpy as np

def run_time_llm_forecast(ticker: str, as_of_date: str, tau: float = -0.0975):
    """
    Time-LLM V1 엔진을 사용해
      - 입력: ticker(문자열 6자리), 기준일(as_of_date, "YYYY-MM-DD")
      - 출력: 10일 예측 경로 + 누적 수익률 + 실제 가격 경로 등
    을 반환하는 헬퍼 함수.

    전제:
      - data_handler: DataHandler 인스턴스
      - configs    : Configs 인스턴스 (seq_len=120, pred_len=10)
      - model      : TimeLLM(configs) + weight 로딩 완료
      - device     : torch.device(...)
      - DataHandler는 close_log까지 StandardScaler로 표준화된 상태
    """

    model.to(device)
    model.eval()

    seq_len  = configs.seq_len
    pred_len = configs.pred_len

    # 1) 날짜 / 스케일된 데이터 가져오기
    as_of_ts = pd.to_datetime(as_of_date)

    df_scaled = data_handler.get_scaled_data_by_ticker(ticker)
    df_scaled = df_scaled.sort_index()

    if len(df_scaled) < seq_len:
        raise ValueError(f"[run_time_llm_forecast] ticker {ticker} 전체 길이가 {len(df_scaled)}로 "
                         f"seq_len={seq_len}보다 짧습니다.")

    # as_of_date가 영업일이 아닐 수 있으니, 그 이전 가장 가까운 날짜로 맞춰주기
    if as_of_ts not in df_scaled.index:
        candidates = df_scaled.index[df_scaled.index <= as_of_ts]
        if len(candidates) == 0:
            raise ValueError(f"[run_time_llm_forecast] {as_of_date} 이전 데이터가 없습니다.")
        as_of_ts = candidates[-1]  # 가장 최근 영업일

    idx = df_scaled.index.get_loc(as_of_ts)

    if idx + 1 < seq_len:
        raise ValueError(
            f"[run_time_llm_forecast] {ticker} @ {as_of_ts.date()} 기준으로 "
            f"필요한 과거 {seq_len}일이 부족합니다. (현재 {idx+1}일)"
        )

    start = idx - seq_len + 1
    end   = idx + 1               # iloc에서 end는 exclusive, 그래서 +1
    window_scaled = df_scaled.iloc[start:end].values  # (seq_len, C)

    # 2) 텐서 변환
    x = torch.FloatTensor(window_scaled).unsqueeze(0).to(device)  # (1, seq_len, C)
    B, Seq, C = x.shape
    Pred = pred_len

    dummy_mark_enc = torch.zeros(B, Seq, 4, device=device)
    dummy_mark_dec = torch.zeros(B, Pred, 4, device=device)
    dummy_dec_in   = torch.zeros(B, Pred, C, device=device)

    # 3) 모델 추론 (스케일된 공간)
    with torch.no_grad():
        out = model(x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
        if isinstance(out, tuple):
            out = out[0]
        preds_scaled = out[:, -pred_len:, :]  # (1, Pred, C)

    # -----------------------------
    # 4) 스케일된 close_log 기반 정보
    # -----------------------------
    pred_close_scaled = preds_scaled[0, :, 0].cpu().numpy()  # (Pred,)
    last_close_scaled = x[0, -1, 0].cpu().item()
    prev_close_scaled = x[0, -2, 0].cpu().item()

    # (1) "학습 시 사용하던" CUM10 (표준화 공간)
    pred_cum10_scaled = float(pred_close_scaled[-1] - last_close_scaled)
    last_ret_scaled   = last_close_scaled - prev_close_scaled
    naive_cum10_scaled = float(last_ret_scaled * Pred)

    # τ 기준으로 방향 레이블 (학습 시와 동일한 공간에서 판정)
    direction_label = "UP_or_NEUTRAL" if pred_cum10_scaled > tau else "DOWN_RISK"

    # (2) 스케일된 경로에서 일별 수익률 (참고용)
    pred_ret_scaled_path = (pred_close_scaled[1:] - pred_close_scaled[:-1]).tolist()

    # -----------------------------
    # 5) StandardScaler 역변환 (denorm)
    # -----------------------------
    scaler = data_handler.scalers_by_ticker[ticker]

    # 마지막 두 개 관측값 (전부 채널 포함) 역변환
    last_two_scaled = x[0, -2:, :].cpu().numpy()    # (2, C)
    last_two_denorm = scaler.inverse_transform(last_two_scaled)  # (2, C)
    prev_full_denorm = last_two_denorm[0]  # (C,)
    last_full_denorm = last_two_denorm[1]  # (C,)

    prev_close_log_denorm = float(prev_full_denorm[0])  # 0번 채널: close_log
    last_close_log_denorm = float(last_full_denorm[0])

    # 예측 경로 전체 역변환
    preds_scaled_np   = preds_scaled[0].cpu().numpy()            # (Pred, C)
    preds_denorm_full = scaler.inverse_transform(preds_scaled_np)  # (Pred, C)
    pred_close_log_denorm = preds_denorm_full[:, 0]              # (Pred,)

    # log(price) → price 복원 (close_log = log(price) 라는 가정)
    last_price = float(np.exp(last_close_log_denorm))
    prev_price = float(np.exp(prev_close_log_denorm))
    pred_price_path = np.exp(pred_close_log_denorm)              # (Pred,)

    # 10일 log-return / price-return
    pred_cum10_log  = float(pred_close_log_denorm[-1] - last_close_log_denorm)
    naive_cum10_log = float((last_close_log_denorm - prev_close_log_denorm) * Pred)

    pred_cum10_return  = float(pred_price_path[-1] / last_price - 1.0)
    naive_cum10_return = float((last_price / prev_price) ** Pred - 1.0)

    result = {
        "ticker": ticker,
        "as_of_date": as_of_ts.strftime("%Y-%m-%d"),
        "pred_horizon_days": Pred,
        "tau_threshold_scaled": tau,

        # --- 모델 학습 시 사용하던 "표준화 공간" 기준 정보 ---
        "scaled_space": {
            "last_close": last_close_scaled,
            "pred_path_close": pred_close_scaled.tolist(),
            "pred_ret_path": pred_ret_scaled_path,
            "pred_cum10": pred_cum10_scaled,
            "naive_cum10": naive_cum10_scaled,
            "direction_label": direction_label,
        },

        # --- 실제 feature 공간 (denorm) ---
        "denorm": {
            "last_close_log": last_close_log_denorm,
            "prev_close_log": prev_close_log_denorm,
            "pred_close_log_path": pred_close_log_denorm.tolist(),

            "last_close_price": last_price,
            "prev_close_price": prev_price,
            "pred_price_path": pred_price_path.tolist(),

            "pred_cum10_log": pred_cum10_log,
            "naive_cum10_log": naive_cum10_log,
            "pred_cum10_return": pred_cum10_return,      # ≈ 10일 수익률
            "naive_cum10_return": naive_cum10_return,
        },
    }

    return result


In [12]:
res = run_time_llm_forecast("010140", "2023-06-30")  # 있는 날짜 대충 하나
res.keys(), res["scaled_space"]["direction_label"], res["denorm"]["pred_cum10_return"]


(dict_keys(['ticker', 'as_of_date', 'pred_horizon_days', 'tau_threshold_scaled', 'scaled_space', 'denorm']),
 'UP_or_NEUTRAL',
 0.08260738849639893)

In [1]:
import torch
torch.cuda.empty_cache()
# 필요하면
del model

NameError: name 'model' is not defined